In [ ]:
!pip install torch torchvision matplotlib

import tensorflow as tf
from torchvision import models, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np



In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize images to 299x299 pixels, needed for Inception V3
    transforms.ToTensor(),  # Convertin images to tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Adjusting pixel values to between -1 and 1
])

# Load CIFAR-10 dataset
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Dataset loaded.")



100%|██████████| 170M/170M [00:12<00:00, 13.3MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Dataset loaded.


In [ ]:
# Load pre-trained Inception V3 moel
inception = models.inception_v3(pretrained=True)

inception.fc = nn.Linear(2048, 10)  # Adjust final layer for 10 classes

inception = inception.cuda()

inception.aux_logits = False  # Disable auxiliary classifier to prevent errors during training

# Load Pre-Trained ResNet50
resnet = models.resnet50(pretrained=True)

resnet.fc = nn.Linear(2048, 10)  # Adjust final layer for 10 classes

resnet = resnet.cuda()

print("Models loaded and customized for CIFAR-10.")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 222MB/s] 
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated si

Models loaded and customized for CIFAR-10.


In [ ]:
def train(model, loader, criterion, optimizer, epochs=1):
    model.train()  # Set the model to training mode

    for epoch in range(epochs):  # Loop over the specified number of epochs
        running_loss = 0.0  # Initialize a variable to accumulate the loss for the current epoch

        for inputs, labels in loader:
            inputs, labels = inputs.cuda(), labels.cuda()  # Move inputs and labels to the GPU

            # Zero the parameter gradients to avoid accumulation from previous steps
            optimizer.zero_grad()

            # Forward pass: compute model predictions
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backward pass: compute gradients
            loss.backward()

            # Update model parameters using the computed gradients
            optimizer.step()

            # Accumulate the loss for this batch
            running_loss += loss.item()

        # Print the average loss for the current epoch
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(loader)}")

def evaluate(model, loader):
    model.eval()  # Set the model to evaluation mode

    #Initialize counters
    correct = 0
    total = 0

    with torch.no_grad():  # No need for backpropagate in eval
        for inputs, labels in loader:
            inputs, labels = inputs.cuda(), labels.cuda()

            # Forward pass
            outputs = model(inputs)

            # Get the predicted class by taking the index of the maximum output (i.e., the predicted class)
            _, predicted = torch.max(outputs.data, 1)

            # Update counters for total labels and correct predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate accuracy as the percentage of correct predictions
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")

    return accuracy


In [ ]:
criterion = nn.CrossEntropyLoss()
# Set the loss function.

optimizer_inception = optim.Adam(inception.parameters(), lr=0.001)
# Uses Adam optimizer for efficiency. adjusts LR based on graients to aid fast convergence.

print("Training Inception V3...")

train(inception, train_loader, criterion, optimizer_inception, epochs=1)
# Call our train function to train the Inception V3 model using the CIFAR-10 training data.

print("Evaluating Inception V3...")

evaluate(inception, test_loader)
# Call our evaluate function to test the trained Inception V3 model on the CIFAR-10 test data.


Training Inception V3...
Epoch 1, Loss: 0.7719107317623235
Evaluating Inception V3...
Accuracy: 82.53%


82.53

In [ ]:
optimizer_resnet = optim.Adam(resnet.parameters(), lr=0.001)

print("Training ResNet50...")
train(resnet, train_loader, criterion, optimizer_resnet, epochs=1)
print("Evaluating ResNet50...")
evaluate(resnet, test_loader)

Training ResNet50...
Epoch 1, Loss: 1.1048722521128445
Evaluating ResNet50...
Accuracy: 65.25%


65.25

In [ ]:
import time

# Measure Inference Time
def measure_inference_time(model, loader):
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        for inputs, _ in loader:
            inputs = inputs.cuda()
            _ = model(inputs)
    print(f"Inference time: {time.time() - start_time:.2f} seconds")

print("Inception V3 Inference Time:")
measure_inference_time(inception, test_loader)

print("ResNet50 Inference Time:")
measure_inference_time(resnet, test_loader)


Inception V3 Inference Time:
Inference time: 23.54 seconds
ResNet50 Inference Time:
Inference time: 21.51 seconds
